In [1]:
__author__ = 'ShuoData: https://www.kaggle.com/shuodata'

#just ignore the warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import sys
import visuals as vs
import renders as rs
from library import *
from cleandf import *
from someclf import *




/home/joaquin/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/joaquin/anaconda2/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
df_test = Load_DataFrames('/Data/total.xlsx','DF_2')
df_train = Load_DataFrames('/Data/total.xlsx','DF_1')

In [3]:
cols = ['people_id', 'activity_id', 'min_date', 'max_date', 'apg', 'ppg', 'date_t', 'wd_t', 'acts', 'group_1',\
        'wd', 'date','char_1_t', 'char_2_t', 'char_3_t', 'char_4_t', 'char_5_t', 'char_6_t', 'char_7_t',\
        'char_8_t', 'char_9_t', 'char_10_t', 'act', 'ac_1', 'ac_2', 'ac_3', 'ac_4', 'ac_5', 'ac_6', 'ac_7',  \
        'char_1', 'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9', 'char_10', \
        'char_11', 'char_12', 'char_13', 'char_14', 'char_15', 'char_16', 'char_17', 'char_18', 'char_19', \
        'char_20', 'char_21', 'char_22', 'char_23', 'char_24', 'char_25', 'char_26', 'char_27', 'char_28', \
        'char_29', 'char_30', 'char_31', 'char_32', 'char_33', 'char_34', 'char_35', 'char_36', 'char_37', 'char_38']

In [4]:
df_train = df_train[cols + ['outcome']]
df_test = df_test[cols]

In [5]:
#df_train = df_train[df_train['group_1'] != 17304 ]

In [6]:
print "Extracting features and outcome from DataFrame..."
features = df_train.drop(['outcome','people_id','activity_id'], axis=1)
outcome = df_train['outcome']

df_train_droped = df_train.drop(['people_id','activity_id'], axis=1)

df_test_droped = df_test.drop(['people_id','activity_id'], axis=1)

## Produce a matrix for test data
test_data = df_test.values.tolist()
test_droped = df_test_droped.values.tolist()

Extracting features and outcome from DataFrame...


In [7]:
print(df_train_droped.shape)
print(df_test_droped.shape)

(1048575, 67)
(498687, 66)


In [8]:
X_train, y_train = features, outcome

In [13]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF, TruncatedSVD

from sklearn.feature_selection import SelectKBest, chi2


def feature_red2(__X,__y):

    pipe = Pipeline([
        ('reduce_dim', PCA()),
        ('classify', RandomForestClassifier())
    ])
    
    N_FEATURES_OPTIONS = [8,16, 32]
    criterion=['entropy']
    max_depth=[63]
    max_features=['auto']
    min_samples_split=[2]
    n_estimators=[18]
    n_jobs=[2]
    
    param_grid = [
        {
            'reduce_dim': [PCA(iterated_power=7)],
            'reduce_dim__n_components': N_FEATURES_OPTIONS,
            'classify__criterion': criterion,
            'classify__max_depth': max_depth,
            'classify__max_features': max_features,
            'classify__min_samples_split': min_samples_split,
            'classify__n_jobs': n_jobs
        },
        {
            'reduce_dim': [TruncatedSVD(n_iter=7, random_state=42)],
            'reduce_dim__n_components': N_FEATURES_OPTIONS,
            'classify__criterion': criterion,
            'classify__max_depth': max_depth,
            'classify__max_features': max_features,
            'classify__min_samples_split': min_samples_split,
            'classify__n_jobs': n_jobs
        },
        {
            #'reduce_dim': [SelectKBest(chi2)],
            'reduce_dim': [PCA(iterated_power=2)],
            'reduce_dim__n_components': N_FEATURES_OPTIONS,
            'classify__criterion': criterion,
            'classify__max_depth': max_depth,
            'classify__max_features': max_features,
            'classify__min_samples_split': min_samples_split,
            'classify__n_jobs': n_jobs
        },
    ]

    
    grid = GridSearchCV(pipe, n_jobs=2, param_grid=param_grid)

    grid.fit(__X, __y)

    
    return grid.best_estimator_

In [14]:
fr = feature_red2(X_train, y_train)

In [15]:
from sklearn.externals import joblib
joblib.dump(fr, 'PCA-RamdonForest.pkl') 

['PCA-RamdonForest.pkl']

In [16]:
fr.get_params()


{'classify': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=63, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=10, n_jobs=2, oob_score=False, random_state=None,
             verbose=0, warm_start=False),
 'classify__bootstrap': True,
 'classify__class_weight': None,
 'classify__criterion': 'entropy',
 'classify__max_depth': 63,
 'classify__max_features': 'auto',
 'classify__max_leaf_nodes': None,
 'classify__min_impurity_split': 1e-07,
 'classify__min_samples_leaf': 1,
 'classify__min_samples_split': 2,
 'classify__min_weight_fraction_leaf': 0.0,
 'classify__n_estimators': 10,
 'classify__n_jobs': 2,
 'classify__oob_score': False,
 'classify__random_state': None,
 'classify__verbose': 0,
 'classify__warm_start': False,
 'reduce_dim': PCA(copy=True, iterated_power=7, n_components=16, random_

In [17]:
y_pred = fr.predict(df_test_droped)

In [18]:
CreateSubmission(df_test, y_pred,"PCA-RamdonForest")

Saving submission for PCA-RamdonForest at 20161111125850...
